In [37]:
import numpy as np
import pandas as pd
import re
import string
import tensorflow as tf

In [39]:
# read in good news data
df = pd.read_csv("articles.csv")
df.head()

,title,content
0,Butler University Creates 2-Year Debt-Free Col...,Butler University of Indianapolis has created ...
1,"Since Pandemic Closed His Business, New Jersey...",The owner of a New Jersey frame shop has been ...
2,PayPal Commits Over $500 Million to Support Mi...,PayPal yesterday announced a $530 million comm...
3,"9-Year-Old and Friends Have Raised $100,000 fo...",Some unlikely heroes in Minneapolis have raise...
4,Hanes is Equipping America’s Homeless With 1 M...,Hanes basic apparel is not only encouraging Am...


In [40]:
# clean up the content from the good news articles
def clean_news(input_data):
    """
    This function transforms the text to all lowercase letters and removes special punctuation
    input: text data
    output: standardized string
    """
    lowercase = tf.strings.lower(input_data)
    no_punctuation = tf.strings.regex_replace(lowercase,
                                  '[%s]' % re.escape(string.punctuation),'')
    return no_punctuation.numpy().decode('utf-8')

In [42]:
# standardize the content 
df['content'] = df['content'].apply(clean_news)
df.head()

,title,content
0,Butler University Creates 2-Year Debt-Free Col...,butler university of indianapolis has created ...
1,"Since Pandemic Closed His Business, New Jersey...",the owner of a new jersey frame shop has been ...
2,PayPal Commits Over $500 Million to Support Mi...,paypal yesterday announced a 530 million commi...
3,"9-Year-Old and Friends Have Raised $100,000 fo...",some unlikely heroes in minneapolis have raise...
4,Hanes is Equipping America’s Homeless With 1 M...,hanes basic apparel is not only encouraging am...


In [44]:
df.shape

(2487, 2)